### 각 개발자가 카카오 사이트에서 받아야 하는 키 값 - 공유 금지

In [113]:
# 카카오 API 개인 개발자 키 설정 (타인에게 공유하지 않고 이번 프로젝을 위해 사용하거나 각 개발자가 사이트에서 생성한 자신의 키값 사용)
KAKAO_API_KEY = 'c1eeebb20cc2eca5f129e462c714235b'

### 개발하면서 느낀 점
1. 함수 호출 및 성능 문제: 단일 건에 대한 함수 호출은 간단해 보이지만, 다량의 데이터 처리를 요구할 때 최적화 작업이 요구된다.

2. 성능 개선의 필요성: 현재 성능적인 부분은 개선이 필요하다.

3. 코파일럿의 도움과 경험 부족: 코파일럿이 제공하는 코드는 많은 도움이 되지만, 경험 부족으로 인해 코드를 그대로 사용하기에는 어려움이 있었다. 코드를 이해하고 수정하는 과정에서 오류가 발생하는 경우가 많았다.

4. .ipynb 파일 작업의 혼동: .ipynb 파일로 작업할 때, 이전 셀의 결과가 이후 셀에 영향을 미치기 때문에 혼동스러웠다. 이전 코드의 결과가 후속 코드에 영향을 미쳐 예상치 못한 결과를 초래할 수 있다. 각 셀의 독립성을 유지하고, 셀 실행 순서를 신중하게 관리해야 한다.

### 추가 설치 필요

In [ ]:
# !pip install requests
# !pip install aiohttp

In [115]:
import requests
import pyarrow.parquet as arrow_parquet
import pyarrow as pa
import pandas as pd
import numpy as np


## 간단한 데이터 프레임으로 테스트
### 리턴 값 document 예
> [{'address': {'address_name': '서울 강남구 개포동 658-1',
   'b_code': '1168010300',
   'h_code': '1168069000',
   'main_address_no': '658',
   'mountain_yn': 'N',
   'region_1depth_name': '서울',
   'region_2depth_name': '강남구',
   'region_3depth_h_name': '개포4동',
   'region_3depth_name': '개포동',
   'sub_address_no': '1',
   'x': '127.05721239635',
   'y': '37.4761560791019'},
  'address_name': '서울 강남구 개포동 658-1',
  'address_type': 'REGION_ADDR',
  'road_address': {'address_name': '서울 강남구 언주로 3',
   'building_name': '우성6차아파트',
   'main_building_no': '3',
   'region_1depth_name': '서울',
   'region_2depth_name': '강남구',
   'region_3depth_name': '개포동',
   'road_name': '언주로',
   'sub_building_no': '',
   'underground_yn': 'N',
   'x': '127.055925867398',
   'y': '37.4767921771121',
   'zone_no': '06316'},
  'x': '127.05721239635',
  'y': '37.4761560791019'}]

In [116]:
# 주소 넣어서 위도 경도 얻는 함수
def get_lat_lng(address):
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {'query': address}
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        result = response.json()
        if result['documents']:
            address_info = result['documents'][0]['address']
            # documents 리턴값 출력
            # display(result['documents'])
            return address_info['y'], address_info['x']  # 위도(y), 경도(x)
    return None, None

# 예제 데이터프레임 생성
data = {
    '시군구': ['서울특별시 중랑구 중화동', ''],
    '번지': ['450', '서울특별시 강남구 대치동 도곡로93길 23']    
}
df = pd.DataFrame(data)

# 주소 생성
df['주소'] = df['시군구'] + ' ' + df['번지']

# 위도와 경도 컬럼 추가
df['위도y'] = None
df['경도x'] = None

# 각 주소에 대해 위도와 경도 가져오기
for i in range(len(df)):
    address = df.loc[i, '주소']
    lat, lng = get_lat_lng(address)
    df.at[i, '위도y'] = lat
    df.at[i, '경도x'] = lng

# Parquet 파일로 저장
table = pa.Table.from_pandas(df)
arrow_parquet.write_table(table, '../addresses.parquet')

# Parquet 파일 읽기
df_read = arrow_parquet.read_table('../addresses.parquet').to_pandas()
display(df_read)


,시군구,번지,주소,위도y,경도x
0,서울특별시 중랑구 중화동,450,서울특별시 중랑구 중화동 450,37.5971495731568,127.081907230062
1,,서울특별시 강남구 대치동 도곡로93길 23,서울특별시 강남구 대치동 도곡로93길 23,37.5013575539498,127.063355872072


### 데이터 로드

In [117]:
# null값 등 처리가 달라지는 문제 때문에 CSV 파일로 다시 진행했습니다.
train_path = '../train.csv'
test_path  = '../test.csv'
dt = pd.read_csv(train_path)
dt_test = pd.read_csv(test_path)

/tmp/ipykernel_313699/808722576.py:4: DtypeWarning: Columns (16,17,36) have mixed types. Specify dtype option on import or set low_memory=False.
  dt = pd.read_csv(train_path)


In [118]:
# train/test 구분을 위한 칼럼을 하나 만들어 줍니다.
dt['is_test'] = 0
dt_test['is_test'] = 1
concat = pd.concat([dt, dt_test])     # 하나의 데이터로 만들어줍니다.

In [119]:
concat.isnull().sum()

시군구                             0
번지                            227
본번                             75
부번                             75
아파트명                         2136
전용면적(㎡)                         0
계약년월                            0
계약일                             0
층                               0
건축년도                            0
도로명                             0
해제사유발생일                   1121899
등기신청일자                          0
거래유형                            0
중개사소재지                          0
k-단지분류(아파트,주상복합등등)         877273
k-전화번호                     876850
k-팩스번호                     879348
단지소개기존clob                1058958
k-세대타입(분양형태)               876125
k-관리방식                     876125
k-복도유형                     876454
k-난방방식                     876125
k-전체동수                     877207
k-전체세대수                    876125
k-건설사(시공사)                 877637
k-시행사                      877834
k-사용검사일-사용승인일              876259
k-연면적                      876125
k-주거전용면적      

In [120]:
# 데이터 값 처리
# 위 처럼 아무 의미도 갖지 않는 칼럼은 결측치와 같은 역할을 하므로, np.nan으로 채워 결측치로 인식되도록 합니다.
concat['등기신청일자'] = concat['등기신청일자'].replace(' ', np.nan)
concat['거래유형'] = concat['거래유형'].replace('-', np.nan)
concat['중개사소재지'] = concat['중개사소재지'].replace('-', np.nan)
concat['번지'] = concat['번지'].replace(' ', np.nan)
concat['도로명'] = concat['도로명'].replace(' ', np.nan)

In [121]:
concat.isnull().sum()

시군구                             0
번지                            227
본번                             75
부번                             75
아파트명                         2136
전용면적(㎡)                         0
계약년월                            0
계약일                             0
층                               0
건축년도                            0
도로명                          1211
해제사유발생일                   1121899
등기신청일자                    1111271
거래유형                      1086451
중개사소재지                    1090013
k-단지분류(아파트,주상복합등등)         877273
k-전화번호                     876850
k-팩스번호                     879348
단지소개기존clob                1058958
k-세대타입(분양형태)               876125
k-관리방식                     876125
k-복도유형                     876454
k-난방방식                     876125
k-전체동수                     877207
k-전체세대수                    876125
k-건설사(시공사)                 877637
k-시행사                      877834
k-사용검사일-사용승인일              876259
k-연면적                      876125
k-주거전용면적      

In [122]:
# 위에서 결측치가 100만개 이하인 변수들만 골라 새로운 concat_select 객체로 저장해줍니다.
selected = list(concat.columns[concat.isnull().sum() <= 1000000])
concat_select = concat[selected]

In [123]:
concat_select.isnull().sum()     # 결측치가 100만개 초과인 칼럼이 제거된 모습은 아래와 같습니다.
# target변수는 test dataset 개수만큼(9272) 결측치가 존재함을 확인할 수 있습니다.

시군구                            0
번지                           227
본번                            75
부번                            75
아파트명                        2136
전용면적(㎡)                        0
계약년월                           0
계약일                            0
층                              0
건축년도                           0
도로명                         1211
k-단지분류(아파트,주상복합등등)        877273
k-전화번호                    876850
k-팩스번호                    879348
k-세대타입(분양형태)              876125
k-관리방식                    876125
k-복도유형                    876454
k-난방방식                    876125
k-전체동수                    877207
k-전체세대수                   876125
k-건설사(시공사)                877637
k-시행사                     877834
k-사용검사일-사용승인일             876259
k-연면적                     876125
k-주거전용면적                  876170
k-관리비부과면적                 876125
k-전용면적별세대현황(60㎡이하)        876170
k-전용면적별세대현황(60㎡~85㎡이하)    876170
k-85㎡~135㎡이하              876170
k-수정일자                    876170
고용보험관리번호  

In [124]:
xy_concat_select = concat_select[['시군구','번지','도로명','좌표X','좌표Y']]
xy_concat_select.head()
xy_concat_select.isna().sum()

시군구         0
번지        227
도로명      1211
좌표X    876232
좌표Y    876232
dtype: int64

### 주소 데이터 결측치 처리
1. 시군구는 Null값 없음
2.  번지가 Null인 모든 데이터가 도로명 있음
3. [ 시군구+도로명 ] 우선으로 위도/경도 결측치 채우고 
4. 도로명이 Null이면 [ 시군구+번지 ]로 결측치 채우기
5. 위도와 경도 가져오기: get_lat_lng 함수를 호출하여 위도와 경도를 가져옵니다.

In [125]:
# 결측치 첫번째 데이터 확인
print(xy_concat_select[xy_concat_select['좌표X'].isna()].head(1))

               시군구   번지     도로명  좌표X  좌표Y
975  서울특별시 강남구 개포동  189  삼성로 14  NaN  NaN


In [126]:
# 최소한의 데이터만 가지고 작업
xy_concat_select.head()

,시군구,번지,도로명,좌표X,좌표Y
0,서울특별시 강남구 개포동,658-1,언주로 3,127.05721,37.476763
1,서울특별시 강남구 개포동,658-1,언주로 3,127.05721,37.476763
2,서울특별시 강남구 개포동,658-1,언주로 3,127.05721,37.476763
3,서울특별시 강남구 개포동,658-1,언주로 3,127.05721,37.476763
4,서울특별시 강남구 개포동,658-1,언주로 3,127.05721,37.476763


### 주소 컬럼 새로 만들어서 값 세팅

In [129]:
df = xy_concat_select

# 주소 생성: '도로명' 있는 경우에는 '시군구+도로명'을 사용하고 '도로명' 없는 경우 '시군구+번지'로 주소 세팅
df['주소'] = np.where(df['도로명'].notna() & df['도로명'].str.strip().astype(bool),
                      df['시군구'] + ' ' + df['도로명'],
                      df['시군구'] + ' ' + df['번지'].fillna(''))
df.head()

/tmp/ipykernel_313699/1925403011.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['주소'] = np.where(df['도로명'].notna() & df['도로명'].str.strip().astype(bool),


,시군구,번지,도로명,좌표X,좌표Y,주소
0,서울특별시 강남구 개포동,658-1,언주로 3,127.05721,37.476763,서울특별시 강남구 개포동 언주로 3
1,서울특별시 강남구 개포동,658-1,언주로 3,127.05721,37.476763,서울특별시 강남구 개포동 언주로 3
2,서울특별시 강남구 개포동,658-1,언주로 3,127.05721,37.476763,서울특별시 강남구 개포동 언주로 3
3,서울특별시 강남구 개포동,658-1,언주로 3,127.05721,37.476763,서울특별시 강남구 개포동 언주로 3
4,서울특별시 강남구 개포동,658-1,언주로 3,127.05721,37.476763,서울특별시 강남구 개포동 언주로 3




### 주의 : 많은 시간 소요 (15분 정도 걸림)
* 테스트를 위해서는 ''' ''' 주석 풀고 실행
* 트레인과 테스트 파일 합친 데이터프레임을 for 루프 돌면서 좌표 없는 값들만 채워준다.
* 동일 주소가 많아서 이전 주소와 같으면 그 주소로 세팅하고 다르면 리모트 함수를 호출하도록 했다.
* 실행없이 결과파일을 사용하거나 확인하려면 함께 업로드한 outputFinal.csv 참조


In [ ]:
'''
# 카카오 함수 호출 정의
def get_lat_lng(address):
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {'query': address}
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        result = response.json()
        if result['documents']:
            address_info = result['documents'][0]['address']
            # documents 리턴값 출력
            # display(result['documents'])
            return address_info['y'], address_info['x']  # 위도(y), 경도(x)
    return None, None

# 유효한 주소에 대해 위도와 경도 가져오기
pre_addr = None
pre_lat, pre_lng = None, None

for i, row in df.iterrows():    
    if pd.isna(row['좌표X']):    
        if address == pre_addr:
            lat, lng = pre_lat, pre_lng        
        else:
            # display('호출')
            lat, lng = get_lat_lng(address)
            pre_addr = address
            pre_lat, pre_lng = lat, lng                            
        
        # 값 세팅
        df.at[i,'좌표Y'] = lat       
        df.at[i,'좌표X'] = lng

display(df)

# # 결과 데이터프레임을 CSV 파일로 저장 
df.to_csv('../outputFinal.csv', index=False) 
print("CSV 파일 저장이 완료되었습니다.")
'''

/tmp/ipykernel_313699/3284368956.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[i,'좌표Y'] = lat
/tmp/ipykernel_313699/3284368956.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[i,'좌표X'] = lng


,시군구,번지,도로명,좌표X,좌표Y,주소
0,서울특별시 강남구 개포동,658-1,언주로 3,127.05721,37.476763,서울특별시 강남구 개포동 언주로 3
1,서울특별시 강남구 개포동,658-1,언주로 3,127.05721,37.476763,서울특별시 강남구 개포동 언주로 3
2,서울특별시 강남구 개포동,658-1,언주로 3,127.05721,37.476763,서울특별시 강남구 개포동 언주로 3
3,서울특별시 강남구 개포동,658-1,언주로 3,127.05721,37.476763,서울특별시 강남구 개포동 언주로 3
4,서울특별시 강남구 개포동,658-1,언주로 3,127.05721,37.476763,서울특별시 강남구 개포동 언주로 3
...,...,...,...,...,...,...
9267,서울특별시 중랑구 신내동,816,신내역로1길 85,127.063355872072,37.5013575539498,서울특별시 중랑구 신내동 신내역로1길 85
9268,서울특별시 중랑구 신내동,816,신내역로1길 85,127.063355872072,37.5013575539498,서울특별시 중랑구 신내동 신내역로1길 85
9269,서울특별시 중랑구 신내동,816,신내역로1길 85,127.063355872072,37.5013575539498,서울특별시 중랑구 신내동 신내역로1길 85
9270,서울특별시 중랑구 신내동,816,신내역로1길 85,127.063355872072,37.5013575539498,서울특별시 중랑구 신내동 신내역로1길 85


CSV 파일 저장이 완료되었습니다.


In [135]:
# 데이터 확인
display(len(df))
display(df['좌표X'].isna().sum())

1128094

0